### IMPORTING LIBRARIES

In [404]:
import pandas as pd

### MAIN DATASET OF ENERGY IMPORTATIONS AND EXPORTATIONS PER CANADIAN PROVINCE

In [395]:
# Getting the monthly energy importations and exportations dataset
path_monthly = 'C:\\Users\\felip\\JupyterLab Projects\\Analytics Tools and Decision Making\\electricity-exports\\electricity-exports-and-imports-monthly.csv'
df = pd.read_csv(path_monthly, sep=',', encoding='ISO-8859-1')

# Taking off the "Total" rows (represents the summation of all energy exported or imported in all provinces in the month)
df = df[df['Source'] != 'Total']
df = df[df['Destination'] != 'Total']

### ADDING PRIME MINISTERS

In [395]:
# Loading the xlsx file with prime ministers mandate
pm = pd.read_excel(r"C:\Users\felip\OneDrive\Documentos\ST CLAIR COLLEGE\First Semester\ANALYTI TOOLS & DECISN MAKING\Project 1\canada_prime_ministers.xlsx")

# Replacing the sign "_x000D_\n" in the dataframe
pm = pm.replace('_x000D_\n', ' ', regex=True)
pm = pm.rename(columns=lambda x: x.replace('_x000D_\n', ' '))

# Droping duplicates (each prime minister name was appearing twice)
pm.drop_duplicates(subset=['Name (Birth–Death)'], keep='first', inplace=True)

# Droping useless columns and reseting index
pm.drop('No.', axis=1, inplace=True)
pm.reset_index(inplace=True, drop=True)

# Creating the "begin" and "end" columns referent to each prime minister mandate
pm['begin'] = pd.to_datetime(pm['Term of office'], format='%d %B %Y', errors='coerce')
pm['end'] = pd.to_datetime(pm['Term of office_1'], format='%d %B %Y', errors='coerce')

# Droping useless columns again
pm.drop(['Term of office', 'Term of office_1', 'Electoral mandates (Assembly)', 'Political party'], axis=1, inplace=True)

# Adding the last day of the dataset analysed (it was empty before)
pm.loc[22,'end'] = pd.datetime(2022, 12, 31)

# This "pm" (prime ministers) final dataset was used to assign the current prime minister and political party for each row of the first dataset "monthly",
# the process was made with MS Excel

### ADDING POPULATION OF EACH PROVINCE

In [395]:
# Loading the csv file with population per Canadian Province
pop = pd.read_csv(r"C:\Users\felip\JupyterLab Projects\Analytics Tools and Decision Making\population_per_province.csv", sep=',')

# The initial dataset had the population per quarter, but we decided to get the population per year, so we left only the fourth quarter pop.
rows_off = pop[pop['Geography'].str.startswith(('Q1', 'Q2', 'Q3'))].index
pop.drop(rows_off, inplace=True)

# Removing the "Q4 " in front of the year, leaving just the year in each row 
pop['Geography'] = pop['Geography'].apply(lambda x: x[3:])

# Standardizing the name of the columns to match the main dataset, reseting index and seting 'Year' as index
pop.columns = ['Year', 'Canada', 'Newfoundland and Labrador', 'Prince Edward Island',
       'Nova Scotia', 'New Brunswick', 'Québec', 'Ontario', 'Manitoba',
       'Saskatchewan', 'Alberta', 'British Columbia', 'Yukon',
       'Northwest Territories', 'Nunavut']
pop.reset_index(drop=True, inplace=True)
pop.set_index('Year', inplace=True)

# Creating the 'Year' column through the date of each row
df['Year'] = df['Period'].apply(lambda x: x[-4:])

# Separating the main dataset in exportations and exportations
df_exp = df[df['Activity'] == 'Exports']
df_imp = df[df['Activity'] == 'Imports']

# Creating the 'Province Pop' with the population of each Province that is either importing or exporting energy, using the pop dataframe
df_exp.loc[:, 'Province Pop'] = df_exp.apply(lambda x: pop.loc[x['Year'], x['Source']], axis=1)
df_imp.loc[:, 'Province Pop'] = df_imp.apply(lambda x: pop.loc[x['Year'], x['Destination']], axis=1)

# Concatenating both dataframes into one again
df = pd.concat([df_exp, df_imp], axis=0)

### ADDING NUMBER OF POWER PLANTS PER PROVINCE

In [395]:
# Loading the csv file with number of power plants per Canadian Province
pp = pd.read_csv(r"C:\Users\felip\OneDrive\Documentos\ST CLAIR COLLEGE\First Semester\ANALYTI TOOLS & DECISN MAKING\Project 1\power_plants.csv")

# Creating a list with all the years analysed
lista = []
for x in range(1990,2023):
    lista.append(x)

# Creating a dictionary with years and respective amount of power plants per province
dic = {'Year':lista}

# For loop to iterate through all the province names and create separate dataframes
for province in pp['Province'].unique():
    lista_pp = []
    filtro = pp[pp['Province'] == province]
    # For loop to iterate through the years analyzed and check how many power plants there were in the province in that year
    for ano in lista:
        lista_pp.append(filtro[filtro['Startup'] < ano].count()['Startup'])
    # Adding the list of number of power plants per year to the dictionary
    dic[province] = lista_pp

# Using the dictionary to create the dataframe
pp_per_year = pd.DataFrame(dic).set_index('Year')

# Standardizing the name of the columns to match the main dataset and reseting index
pp_per_year.columns = ['Ontario', 'Québec', 'Newfoundland and Labrador', 'British Columbia',
       'Alberta', 'Manitoba', 'New Brunswick', 'Saskatchewan', 'Nova Scotia']
pp_per_year.index = pp_per_year.index.astype(str)

# Creating the 'Province Power Plants' with the number of power plants in the province that is either importing or exporting
# energy, using the pp_per_year dataframe
df_exp.loc[:, 'Province Power Plants'] = df_exp.apply(lambda x: pp_per_year.loc[x['Year'], x['Source']], axis=1)
df_imp.loc[:, 'Province Power Plants'] = df_imp.apply(lambda x: pp_per_year.loc[x['Year'], x['Destination']], axis=1)

# Concatenating both dataframes into one again
df = pd.concat([df_exp, df_imp], axis=0)

### ADDING THE GDP PER PROVINCE

In [395]:
# Loading the csv file with the GDP per Canadian Province
gdp = pd.read_csv(r"C:\Users\felip\OneDrive\Documentos\ST CLAIR COLLEGE\First Semester\ANALYTI TOOLS & DECISN MAKING\Project 1\gdp from 1990.csv", index_col=0)

# Setting 'Year' as the index name
gdp.index.name = 'Year'

# Standardizing the name of the columns to match the main dataset
gdp.columns = ['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Québec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories', 'Nunavut']

# Setting the index type as string
gdp.index = gdp.index.astype(str)

# Using the gdp of 2021 to 2022 (We didn't find the 2022 GDP in the dataset)
gdp.loc['2022',] = gdp.loc['2021',]

# Creating the 'Province GDP' with the GDP of each Province that is either importing or exporting energy, using the gdp dataframe
df_exp.loc[:, 'Province GDP'] = df_exp.apply(lambda x: gdp.loc[x['Year'], x['Source']], axis=1)
df_imp.loc[:, 'Province GDP'] = df_imp.apply(lambda x: gdp.loc[x['Year'], x['Destination']], axis=1)

# Concatenating both dataframes into one again, and reseting index of the main dataframe
df = pd.concat([df_exp, df_imp], axis=0)
df.reset_index(drop=True, inplace=True)